# Web Scraping

Poniższy kod pobiera ze strony Gumtree dane odnośnie sprzedaży mieszkań w Warszawie. W projekcie autorka wykorzystała bibliotekę BeautifulSoup. Do rozwiązania problemu z serwerem proxy wykorzystano instrukcję ze strony: https://pl.admininfo.info/no-se-ha-podido-establecer-conexi-n-con-el-servidor-proxy-windows-10

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib
import time

In [2]:
url='https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/v1c9073l3200008a1dwp1?priceType=FIXED'


npo_tags = {}
tag_no = 0

while True:

    #Rozwiązanie problemu z serwerem proxy
    headers = {"User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.2.8) Gecko/20100722 Firefox/3.6.8 GTB7.1 (.NET CLR 3.5.30729)", "Referer": "http://example.com"}
    response = requests.get(url, headers=headers, proxies=urllib.request.getproxies(), timeout=5)
    data=response.text
    soup=BeautifulSoup(data, 'html5lib')        
    tags = soup.find_all('div',{'class':'tileV1'})
    
   
    for tag in tags:
            
        link = tag.find('a', {'class': 'href-link'}).get('href') 
        link='https://www.gumtree.pl' + link
        tag_response = requests.get(link)
        tag_data = tag_response.text
        tag_soup = BeautifulSoup(tag_data, 'html5lib') 
            
        #Rozwiązanie problemu, gdy find zwraca None
        node = tag_soup.find('span',{'class':'pre'})
        if node is not None:
            tag_description = tag_soup.find('span',{'class':'pre'}).text
        else:
            tag_description = node
                
        node1=tag_soup.find('span',{'class':'amount'})
        if node1 is not None:
            cena=tag_soup.find('span',{'class':'amount'}).text
        else:
            cena=node1
            
        #Wybór wszystkich wypełnionych pól odnośnie mieszkania
        name_obj=tag_soup.find_all(string=["Data dodania", "Lokalizacja", "Na sprzedaż przez",
        "Rodzaj nieruchomości", "Liczba pokoi", "Liczba łazienek", "Wielkość (m2)", "Parking"])
        #Wybranie tylko unikatowych wartości
        name_obj=list(set(name_obj))
        
            
        i=0
        while i<len(name_obj):
                

            if 'Data dodania' in name_obj:  
                data_dodania = tag_soup.find('div',{'class':'attribute'}).text[12:22]
                i += 1
                
                
            #Pobranie tylko dzielnicy miasta
            if 'Lokalizacja' in name_obj:  
                lokalizacja=tag_soup.find_all('div',{'class':'attribute'})[i].text[11:].split(',')[0] 
                i += 1
                   

            if 'Na sprzedaż przez' in name_obj: 
                na_sprzedaż_przez=tag_soup.find_all('div',{'class':'attribute'})[i].text[17:]
                i += 1
                
            
            if 'Rodzaj nieruchomości' in name_obj: 
                rodz_nieruchomości=tag_soup.find_all('div',{'class':'attribute'})[i].text[20:] 
                i += 1
                

            if 'Liczba pokoi' in name_obj: 
                l_pok=tag_soup.find_all('div',{'class':'attribute'})[i].text[12:] 
                i += 1
                
                
            if 'Liczba łazienek' in name_obj: 
                l_laz=tag_soup.find_all('div',{'class':'attribute'})[i].text[15:] 
                i += 1
                
                
            if 'Wielkość (m2)' in name_obj: 
                wielk=tag_soup.find_all('div',{'class':'attribute'})[i].text[13:] 
                i += 1
                
                
            if 'Parking' in name_obj:   
                parking=tag_soup.find_all('div',{'class':'attribute'})[i].text[7:]
                i += 1


        #Zapisanie danych i wydrukowanie ich
        tag_no+=1
        npo_tags[tag_no] = [link, cena, lokalizacja, wielk , data_dodania, na_sprzedaż_przez, rodz_nieruchomości, l_pok, l_laz, parking, tag_description]
            

        print('\nLink:', link)
        
    url_tag = soup.find('a',{'class':'arrows icon-right-arrow icon-angle-right-gray'}, href=True)
    if url_tag:
        url= 'https://www.gumtree.pl' + url_tag.get('href')
        print(url)
    else:
        break

        
print("Total tags:", tag_no)


npo_tags_df = pd.DataFrame.from_dict(npo_tags, orient = 'index', columns = ['Link', 'Cena', 'Lokalizacja', 'Wielkość (m^2)', 'Data Dodania', 'Na sprzedaż przez', 'Rodzaj nieruchomości', 'Liczba pokoi', 'Liczba łazienek', 'Parking', 'Opis'])


npo_tags_df.head()


Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/srodmiescie-piekne-2-pokojowe-mieszkanie-56m2-ul-andersa-24/1007734340630910744291509

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/ochota/nowa-cena-dwupokojowe-mieszkanie-na-pradze-polnoc/1008180859740912476393509

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/3-pokoje-w-bardzo-atrakcyjnej-cenie/1008114432430910762815009

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/nowoczesny-pelen-pasji-apartament-z-ogrodem/1008243742730912530316909

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/ustawne-2-pokoje-do-wprowadzenia-metro/1008243809480912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wilanow/3+pokojowe-na-wilanowie-+super-cena/1008243887060912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/0-z-potencjalem-w-b-dobrej-lokalizac


Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/przestronne-mieszkanie-srodmiescie-ul-ciasna/1008243809050911378860309

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wola/1-pokoj-wola-ul-wolska/1008243772130911378860309

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/3-pokoje-%252B-oddzielna-kuchnia-blisko-parku/1008243772120912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/3-pokoje-praga+poludnie-ul-ostrobramska/1008243772110911378860309

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wilanow/3-pokoje-wilanow-ul-hlonda/1008243809040911378860309

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/2-pokojowe-mieszkanie-targowek-ul-wysockiego/1008243809030911378860309

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wlochy/1-pokojowe-mieszkanie-wlochy-ul-nike/1008243809010911378860309

Link: https://


Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/3-pokoje-w-bardzo-atrakcyjnej-cenie/1008114432430910762815009

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/saska-kepa-zwyciezcow-48m2-po-remoncie-parking-bezposrednio/1008228924860911209431409

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wola/nowa-urzadzona-kawalerka-warszawa-wola/1008200387430912554401509

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/2+pokojowe-przy-metro-politechnika/1008243679420912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wlochy/4-niezalezne-pokoje-w-super-lokalizacji/1008243695340912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/4-duze-pokoje-na-targowku/1008243695330912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/dom-z-potencjalem-w-prestizowej-lokalizacji/1008243399460912592146909

Lin


Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/niesamowity-odremontowany-apartament-z-1885r-centrum-84-m2/1008243536880912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bielany/4-pokoje-63mkw-bielany/1008243679030912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wola/dwa-pokoje-obok-metro/1008243589480912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/najlepsze-4-pokoje-w-tej-cenie/1008243459750912559440109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/pieknie-urzadzone-gotowe-do-zamieszkania/1008243410880912534354409

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/4-pokoje-blisko-ul-sobieskiego/1008243589450912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/ursynow/nowoczesny-dom-na-ursynowie/1008243337830912534354409

Link: https://www.gumtree.pl/a-mieszkania-i-domy-


Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bielany/jasne-+2-pokoje-+bielany/1008243440910912407250109
https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-11/v1c9073l3200008a1dwp11?priceType=FIXED

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/ochota/piekne-2-pokojowe-przy-parku-szczesliwickim/1008213215350910670976609

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wola/prawdopodobnie-najlepsze-nowe-m-na-woli-sprawdz/1007902836960911340518009

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/2-pokoje-kamienica-51m2-+-chelmska-mokotow-sielce-kw-+-mozliwosc-kredytowania/1008144704200910942961709

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/rembertow/dwupokojowe-w-rembertowie/1008243563160912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wlochy/taras-56-m2-i-3-pokoje/1008243536320912407250109

Link: https://www.gumtree

...


Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wola/nowoczesny-apartament-na-woli/1008242434270912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wilanow/3+pokojowe+wilanow-+okazja/1008242449220912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bemowo/przestronne-mieszkanie-garaz-w-cenie/1008242449200912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/apartament-gorny-mokotow/1008242449190912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/kawalerka-w-centrum-przy-metrze-swietokrzyska/1008242449170912407250109
https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-42/v1c9073l3200008a1dwp42?priceType=FIXED

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wilanow/3+pokojowe+wilanow-+okazja/1008242449220912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bemowo/p


Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bielany/plac-konfederacji-+-2-pok-ul-schroegera-+-metro/1008241826740912506370709

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/ursynow/mieszkanie-na-kabatach-na-sprzedaz/1008242291540911366864809

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bialoleka/atrakcyjne-mieszkanie-na-tarchominie/1008242252750911366864809

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/penthouse-przy-torze-stegny-taras-86-m2/1008242257610911366864809

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/3-pokoje-na-sadybie-+10-min-do-metra-i-tramwaju/1008242291530912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/3-pokoje-do-remontu-w-srodmiesciu/1008242247760912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/5-pokoi-apartament-118m2-targowek-zacisze/1008242291520911366864809

L


Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wola/2-pokoje-z-balkonem-w-centrum/1008242247410912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bemowo/kawalerka-z-tarasem-22-m2-na-bemowie-za-392-000-zl/1008242252420911355524009

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bialoleka/mieszkanie-w-stanie-deweloperskim-na-sprzedaz/1008242291130911366864809

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bialoleka/bezposrednio-mieszkanie+duzy-taras-ul-glebocka/1008241938550910661363309

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/kawalerka+1-pokoj+targowek+blisko-metra/1008242252410912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/kawalerka-na-brodnie-przy-metrze/1008242247400911355524009

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/cybernetyki-6a-gotowe-mieszkanie-20m2-balkon/1008242247390911355524009


Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wola/mieszkanie-warszawa-wola-38m2-nr-24883/1008242252090912101150309

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bemowo/trzy-pokoje-na-nowoczesnym-osiedlu-przy-metrze/1008242164740911355524009

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/ochota/3-pokojowe-kolonia-lubeckiego-ochota-kamienica/1008242257060911355524009

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bialoleka/nowoczesna-kawalerka-na-tarchominie/1008242247100912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/przestronne-3-pokoje-w-swietnej-lokalizacji/1008242164710911355524009

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/3-pokoje-100-m2/1008242252060912407250109

Link: https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/aleja-szucha-165-m2-5-pok-kamienica-z-1930-r/1008242164680911543980109

Link: https://www

,Link,Cena,Lokalizacja,Wielkość (m^2),Data Dodania,Na sprzedaż przez,Rodzaj nieruchomości,Liczba pokoi,Liczba łazienek,Parking,Opis
1,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,770 000 zł,Śródmieście,56,29/07/2020,Właściciel,Mieszkanie,2 pokoje,1 łazienka,Brak,"TYLKO BEZPOŚREDNIO, BEZ AGENCJIPiękne mieszkan..."
2,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,479 999 zł,Ochota,38,09/10/2020,Agencja,Mieszkanie,2 pokoje,1 łazienka,Ulica,"KUPUJĄCY NIE PŁACI PROWIZJI!!Na Pradze Północ,..."
3,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,410 000 zł,Targówek,47,28/09/2020,Agencja,Mieszkanie,3 pokoje,1 łazienka,Ulica,"Bardzo przytulne, ciche mieszkanie 3-pokojow..."
4,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,799 000 zł,Praga Południe,61,20/10/2020,Agencja,Mieszkanie,3 pokoje,1 łazienka,Kryty,Mam ogromną przyjemność zaprezentować Wam nowo...
5,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,399 000 zł,Mokotów,28,20/10/2020,Agencja,Mieszkanie,2 pokoje,1 łazienka,Kryty,Opis lokalu:\nDwupokojowe mieszkanie o powierz...


In [3]:
npo_tags_df

,Link,Cena,Lokalizacja,Wielkość (m^2),Data Dodania,Na sprzedaż przez,Rodzaj nieruchomości,Liczba pokoi,Liczba łazienek,Parking,Opis
1,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,770 000 zł,Śródmieście,56,29/07/2020,Właściciel,Mieszkanie,2 pokoje,1 łazienka,Brak,"TYLKO BEZPOŚREDNIO, BEZ AGENCJIPiękne mieszkan..."
2,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,479 999 zł,Ochota,38,09/10/2020,Agencja,Mieszkanie,2 pokoje,1 łazienka,Ulica,"KUPUJĄCY NIE PŁACI PROWIZJI!!Na Pradze Północ,..."
3,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,410 000 zł,Targówek,47,28/09/2020,Agencja,Mieszkanie,3 pokoje,1 łazienka,Ulica,"Bardzo przytulne, ciche mieszkanie 3-pokojow..."
4,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,799 000 zł,Praga Południe,61,20/10/2020,Agencja,Mieszkanie,3 pokoje,1 łazienka,Kryty,Mam ogromną przyjemność zaprezentować Wam nowo...
5,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,399 000 zł,Mokotów,28,20/10/2020,Agencja,Mieszkanie,2 pokoje,1 łazienka,Kryty,Opis lokalu:\nDwupokojowe mieszkanie o powierz...
...,...,...,...,...,...,...,...,...,...,...,...
1079,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,1 800 000 zł,Śródmieście,97,20/10/2020,Agencja,Mieszkanie,3 pokoje,2 łazienki,Ulica,Opis:\nPrzestronne 3-pokojowe mieszkanie w prz...
1080,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,450 000 zł,Ursus,38,20/10/2020,Agencja,Mieszkanie,2 pokoje,2 łazienki,Ulica,Pragnę Państwu zaprezentować Mieszkanie wykońc...
1081,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,620 000 zł,Wola,59,20/10/2020,Agencja,Mieszkanie,3 pokoje,2 łazienki,Ulica,Zapraszamy do zakupu 3 pokojowego mieszkania n...
1082,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,895 000 zł,Śródmieście,64,20/10/2020,Agencja,Mieszkanie,2 pokoje,1 łazienka,Ulica,ŚCISŁE CENTRUM WARSZAWY - 2-POKOJOWE MIESZKANI...


In [4]:
#Zapisanie danych do pliku csv
npo_tags_df.to_csv('npo_tags_df.csv', sep=';', index = False, header=True)

In [5]:
#Sprawdzenie czy nie ma jakiś nulli
npo_tags_df[npo_tags_df['Cena'].notnull()==False]

,Link,Cena,Lokalizacja,Wielkość (m^2),Data Dodania,Na sprzedaż przez,Rodzaj nieruchomości,Liczba pokoi,Liczba łazienek,Parking,Opis
